# Load review data

In [2]:
import pandas as pd

reviews = pd.read_csv("../data/tmp_reviews.csv", sep=",", index_col=False, encoding="utf-8")

In [3]:
import random
reviews = reviews.sample(n=40)

# Define openAI function for sentiment chat completion
Constant
- model: gpt-4o
- max_tokens: 1
- temperature: 0
What to define in loop
- message: system + user/assistant prompts

Output: response as integer

In [1]:
# load and set API key for openAI

import openai
from openai import OpenAI

openai.api_key = open("C:/Users/munnes/Documents/API_Keys/openai_20240613.txt", "r").read()

client = OpenAI(api_key=openai.api_key)

In [4]:
def sentiment_analysis(message=list):

    # Call the OpenAI API to generate a response
    response = client.chat.completions.create(
        model="gpt-4o",  # Use a powerful model for sentiment analysis
        messages=message,
        max_tokens=1,  # Limit response to a single word
        temperature=0  # Keep response consistent
    )

    # Extract the sentiment from the response
    sentiment = response.choices[0].message.content
    sentiment = int(sentiment)

    return sentiment

In [ ]:
# https://platform.openai.com/docs/guides/text-generation/chat-completions-api

system_short = "Du bist trainiert, die Qualität von deutschsprachigen Büchern aus kurzen Rezensionen zu extrahieren. Bewerte die Qualität des besprochenen Buches auf einer Skala von 1 (sehr schlecht) bis 7 (sehr gut). Antworte nur mit der Zahle."

system_long = "Du bist trainiert, die Qualität von deutschsprachigen Büchern aus kurzen Rezensionen zu extrahieren. Die Rezensionen enthalten oft auch kurze Zusammenfassungen des Inhalts, der für die Bewertung nicht wichtig ist. Fokussiere dich lediglich auf die verschiedenen Bewertungenmaßstäbe, gehe dabei systematisch vor und prüfe, welche Dimensionen der Buchqualität besprochen und bewertet werden. Bewerte die Qualität des besprochenen Buches auf einer Skala von 1 (sehr schlecht) bis 7 (sehr gut). Antworte nur mit der Zahle."

In [5]:
message_zeroshot = [
    {"role": "system", "content": system},
    {"role": "user", "content": review}
  ]

SyntaxError: expression expected after dictionary key and ':' (2124296237.py, line 3)

In [ ]:

message_fewshot_1 = [
    {"role": "system", "content": system},
    {"role": "user", "content": }
  ]

message_fewshot_2 = [
    {"role": "system", "content": system},
    {"role": "user", "content": fewshots["review"][0]},
    {"role": "assistant", "content": fewshots["sent_hc_7"][0]},
    {"role": "user", "content": fewshots["review"][1]},
    {"role": "assistant", "content": fewshots["sent_hc_7"][1]},
    {"role": "user", "content": fewshots["review"][2]},
    {"role": "assistant", "content": fewshots["sent_hc_7"][2]},
    {"role": "user", "content": fewshots["review"][3]},
    {"role": "assistant", "content": fewshots["sent_hc_7"][3]},
    {"role": "user", "content": text}
  ]

In [43]:
sentiments_zeroshot = []

for i, row in reviews.iterrows():
    # if i % 10 == 0: print(i)
    sentiment = sentiment_analysis(row["review"])
    sentiments_zeroshot.append(sentiment)

In [ ]:
reviews["sent_openai_zeroshot"] = sentiments_zeroshot

In [80]:
def random_rating(df, ratings, n):

  revs_examples = pd.DataFrame(columns=['review', 'sent_hc_7'])

  for rating in ratings:
    revs = df[df["sent_hc_7"] == rating]
    revs = revs.sample(n=n)

    revs_examples = pd.concat([revs_examples, revs], ignore_index=True)

  revs_examples = revs_examples.sample(frac=1).reset_index(drop=True)
  
  revs_examples["sent_hc_7"] = revs_examples["sent_hc_7"].astype(str)
  
  return(revs_examples)

In [81]:
reviews_examples = random_rating(reviews, [1, 3, 5, 7], 1)
reviews_examples

,review,sent_hc_7,sent_openai_zero_shot,sent_openai_fewshot1,sent_openai_zeroshot
0,Einfach nur schrecklich findet Rezensentin Kat...,1,7.0,1.0,7.0
1,"Helmut Krausser hat mit ""Eros"" auch ein ""Sitte...",3,6.0,2.0,6.0
2,Mit großem Vergnügen hat Rezensent Burkhard Mü...,7,4.0,6.0,4.0
3,"Die Protagonistin in diesem Roman, den der Fro...",5,6.0,5.0,6.0


In [82]:
reviews_examples["review"][0]

'Einfach nur schrecklich findet Rezensentin Katharina Döbler Durs Grünbeins Poem zur Zerstörung Dresdens, für das er das Bild des Porzellans für ihren Geschmack reichlich überstrapaziert. "Schönheit und Zerbrechlichkeit, Hybris und Untergang" werden so unerbittlich zusammengereimt, dass die Rezensentin sich gelegentlich an die "Kalauer eines blödelnden Fernseh-Entainers" erinnert fühlt ("Von der bella ante bellum - nichts mehr da"). Meist aber sind ihr Bilder und Reimschema einfach zu bieder. Und das "altväterliche, offen patriarchalische Frauenbild" findet sie ebenfalls recht unzeitgemäß. Fragwürdig schließlich kommt ihr vor, was Grünbein daraus folgert, dass die Frauenkirche nach der Bombardierung noch einige Tage stand, bevor sie zusammenbrach: "so wie sie die Haltung wahren."'

In [88]:
sentences = []

for i, row in reviews_examples.iterrows():
    # Construct a sentence using the values from the DataFrame
    sentence1 = f"Text: {row['review']}\n"
    sentence2 = f"Sentiment: {row['sent_hc_7']}\n"
    sentences.append(sentence1 + sentence2)

sentences = "\n".join(sentences)

fewshot1 = 
  "Im Folgenden die Sentiment Analyse der rezensierten Bücher:\n" +
  sentencens +
  ""

print(sentences)

Text: Einfach nur schrecklich findet Rezensentin Katharina Döbler Durs Grünbeins Poem zur Zerstörung Dresdens, für das er das Bild des Porzellans für ihren Geschmack reichlich überstrapaziert. "Schönheit und Zerbrechlichkeit, Hybris und Untergang" werden so unerbittlich zusammengereimt, dass die Rezensentin sich gelegentlich an die "Kalauer eines blödelnden Fernseh-Entainers" erinnert fühlt ("Von der bella ante bellum - nichts mehr da"). Meist aber sind ihr Bilder und Reimschema einfach zu bieder. Und das "altväterliche, offen patriarchalische Frauenbild" findet sie ebenfalls recht unzeitgemäß. Fragwürdig schließlich kommt ihr vor, was Grünbein daraus folgert, dass die Frauenkirche nach der Bombardierung noch einige Tage stand, bevor sie zusammenbrach: "so wie sie die Haltung wahren."
Sentiment: 1

Text: Helmut Krausser hat mit "Eros" auch ein "Sittenbild der alten Bundesrepublik" geschrieben, erklärt Rezensent Rainer Moritz, doch sein eigentliches erzählerisches Anliegen ist individue

In [76]:
sentiments_fewshots1 = []

for i, row in reviews.iterrows():
    print(i)
    sentiment = sentiment_analysis_fewshots(reviews_examples, row["review"])
    sentiments_fewshots1.append(sentiment)

2399
2455
5047
5250
3400
762
1441
3467
935
3781
2553
2984
5646
5130
5783
4582
2641
3884
4852
90
2014
5449
4912
1762
4784
5179
3140
2329
3316
1701
2796
5002
1325
3078
3184
5561
4658
3092
1733
3358


In [78]:
reviews["sent_openai_fewshot1"] = sentiments_fewshots1

In [111]:
reviews.to_csv("reviews_sentiment.csv")

In [ ]:
conda install -c conda-forge pingouin

In [108]:
revs_long = reviews[["sent_hc_7", "sent_openai_fewshot1"]].rename(columns={"sent_hc_7": "rating_human", "sent_openai_fewshot1": "rating_fewhot1"})

revs_long["id"] = revs_long.index

# Ensure the specific columns are of correct dtype.
revs_long['rating_human'] = revs_long['rating_human'].astype(str)
revs_long['rating_fewhot1'] = revs_long['rating_fewhot1'].astype(str)

revs_long = pd.wide_to_long(revs_long, stubnames=['rating'], i='id', j="rating", sep='_', suffix='.*', )

AttributeError: 'DataFrame' object has no attribute 'str'

In [89]:
import pingouin as pg



icc = pg.intraclass_corr(data=data, targets='Wine', raters='Judge',
                         ratings='Scores')

Channels:
 - defaults
Platform: win-64
Solving environment: ...working... failed

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - pingouin

Current channels:

  - defaults

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


